In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = '../data/dogbreeds/'
sz = 224
arch = resnext101_64
bs = 58

In [2]:
def get_data(sz,bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv',
                                        test_name='test', val_idxs=[0],
                                        suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz > 300 else data.resize(340, 'tmp')

In [3]:
data = get_data(sz,bs)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)
learn.fit(1e-1, 2)

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))


100%|██████████| 1/1 [00:00<00:00, 14.29it/s]


HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.700697   4e-06      1.0       
    1      0.486957   0.000154   1.0                         



[0.00015449524, 1.0]

In [4]:
learn.precompute=False

In [ ]:
learn.fit(1e-1, 5, cycle_len=1)
learn.save('224_pre_all')

In [5]:
learn.set_data(get_data(299, bs))
learn.load('224_pre_all')

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [6]:
learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.247205   0.000182   1.0       
    1      0.223273   0.000196   1.0                         
    2      0.211833   0.00022    1.0                         



[0.00022029877, 1.0]

In [7]:
learn.save('299_pre_all')
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.201513   0.000118   1.0       
    1      0.190621   8.2e-05    1.0                         
    2      0.196204   0.000151   1.0                         
    3      0.205026   0.000158   1.0                         
    4      0.185188   0.000158   1.0                         
    5      0.181745   0.000168   1.0                         
    6      0.18033    0.000116   1.0                         



[0.00011634827, 1.0]

In [8]:
learn.save('299_pre_all')
learn.fit(1e-2, 2, cycle_len=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.181055   0.000162   1.0       
    1      0.18432    0.000101   1.0                         
    2      0.20266    0.000179   1.0                         
    3      0.180429   0.000166   1.0                         



[0.00016593933, 1.0]

In [12]:
%%time
#learn.save('299_pre_all')
learn.load('299_pre_all_tmp')
test = learn.predict(is_test=True)

CPU times: user 8min 9s, sys: 28.6 s, total: 8min 38s
Wall time: 7min 15s


In [13]:
columns = pd.read_csv(f'{PATH}sample_submission.csv', index_col='id').columns
test = pd.DataFrame(np.exp(test))
test.columns = columns
test.index = [i.split('.jpg')[0].split('/')[-1] for i in data.test_dl.dataset.fnames]
test.index.name ='id'
test.to_csv(f'{PATH}submission_all_tmp.csv')

In [14]:
!kg submit {PATH}submission_all_tmp.csv -m 'all data less training'

0.21332
